In [ ]:
from flask import Flask, render_template, url_for, request, flash, redirect, jsonify 
from flask_sqlalchemy import SQLAlchemy
import json
import os
from datetime import datetime

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///app.db'
db = SQLAlchemy(app)


# Database Model
class blogInfo(db.Model):
    id = db.Column(db.Integer, primary_key = True)
    content = db.Column(db.String(200), nullable = False)
    date_uploaded = db.Column(db.DateTime, default = datetime.utcnow)

    @property
    def serialize(self):
        return{
            'id': self.id,
            'content': self.content,
            'date_uploaded': self.date_uploaded,
        }

def __repr__(self):
    return '<Task %r>' % self.id

# Add blog post to database
@app.route('/add_blog',methods=["POST"])
def create_blog():
    data = request.get_json()
 
    new_blog=blogInfo(id=data["id"],content=data["content"],date_uploaded=datetime.utcnow)
            
    db.session.add(new_blog)
    db.session.commit()
 
    blog_id = getattr(new_blog, "id")
    return jsonify({"id": blog_id})

#code for retrieving blog--every blog
@app.route('/retr_blogs', methods =["GET"])
def retrieve_blogs():
    blog = blogInfo.query.all()
    serialized_data = []

#-- retrieving blog by a single id
@app.route('/retr_blog/<int:id>', methods=["GET"])
def retr_single_blog(id):
    blog = blogInfo.filter_by(id=id).first()
    serialized_blog = blog.serialize

# saving data to the model
@app.route('/admin',methods =['POST'])
def user_input():
    user_input= blogInfo(request.form['id'], request.form['email'])
    db.session.add(user_input)
    db.session.commit()
    return redirect(url_for('/'))


#Home page
@app.route('/')
def home_page():
    with open('posts.json', 'r') as file:
        posts = json.load(file)["posts"]

    return render_template('index.html',posts_list = posts)

@app.route('/admin',methods=('GET','POST'))
def admin():
    return render_template('admin.html')



# Error Code Handling
@app.errorhandler(404)
def not_found(e):
    return render_template('404.html'), 404

def main():
    if not os.path.exists('app.db'):
        db.create_all()
    app.run()


if __name__ == "__main__":
    main()